https://docs.pytorch.org/tutorials/intermediate/FSDP_tutorial.html 공식 튜토리얼 내용입니다


## Fully Sharded Data Parallel (FSDP2) 시작하기

**생성일:** 2022년 3월 17일 | **마지막 업데이트:** 2025년 9월 2일 | **마지막 검증:** 2024년 11월 5일

**저자:** Wei Feng, Will Constable, Yifan Mao

> **참고**
>
> 이 튜토리얼의 코드는 [pytorch/examples](https://www.google.com/search?q=https://github.com/pytorch/examples/blob/main/distributed/FSDP/FSDP_example.py)에서 확인하세요.
> FSDP1은 더 이상 사용되지 않습니다(deprecated). FSDP1 튜토리얼은 [1]과 [2]에 보관되어 있습니다.

-----



## FSDP2 작동 방식

DDP(DistributedDataParallel) 학습에서는 각 랭크(rank)가 모델 복제본(replica)을 소유하고 데이터 배치를 처리한 다음, `all-reduce`를 사용해 랭크 간에 그래디언트를 동기화합니다.

DDP와 비교할 때, **FSDP는 모델 파라미터, 그래디언트, 옵티마이저 상태를 샤딩(sharding)하여 GPU 메모리 사용량을 줄입니다.** 이를 통해 단일 GPU에 맞지 않는 대규모 모델을 학습할 수 있습니다. 아래 그림과 같습니다.

  * Forward 및 backward 계산 외부에서는 파라미터가 **완전히 샤딩**됩니다.
  * Forward 및 backward 전에, 샤딩된 파라미터는 `all-gather`를 통해 **샤딩되지 않은 파라미터**로 수집됩니다.
  * Backward 내부에서는, 로컬의 샤딩되지 않은 그래디언트가 `reduce-scatter`를 통해 **샤딩된 그래디언트**로 변환됩니다.
  * 옵티마이저는 샤딩된 그래디언트로 샤딩된 파라미터를 업데이트하며, 이는 **샤딩된 옵티마이저 상태**를 생성합니다.

![FSDP](https://docs.pytorch.org/tutorials/_images/fsdp_workflow.png)



FSDP는 DDP의 `all-reduce` 연산을 `reduce-scatter`와 `all-gather` 연산으로 분해한 것으로 간주할 수 있습니다.

![FSDP_Allreduce](https://docs.pytorch.org/tutorials/_images/fsdp_sharding.png)

### FSDP와 DDP의 차이

DDP는 전체의 파라미터를 Allreduce를 통해서 한번에 동기화 시키는 방식을 사용하게 됩니다. 결과적으로 이 '동기화'과정에서 큰 차이가 발생합니다.  

샤딩되어있는 구조를 보면 FSDP에 저장되는 파라미터는 매우 일부에 불과합니다. 그렇다 하더라도 Forward 연산에서는 결국 모든 파라미터가 필요하기 때문에 forward연산에서는 all-gather 를 통해서 다른 GPU들에 샤딩되어있는 파라미터 조각들을 불러옵니다. (적어도 연산에 참여하는 파라미터를 불러옵니다. 예를들어 A,B가 필요하다면 GPU1로부터 B만은 불러와야합니다.)

FSDP에는 명시적인 동기화 시점이 없습니다. 간단히 말하면 FSDP는 전체 parameter set이 동기화 시점에 전부 이동하는게 아닌, 비동기적으로 그때그때 all-gather 해서 돌아가도록 만들어져 있습니다.


이는 예를들어 GPU0에서 (i+1)th layer 에 대한 forward를 하는 시점에 all-gather 하고 있을때는 GPU2 번에서 all gather 하고 있고 GPU1에서는 (i)th layer 대한 backward 를 하고 있기 때문에 all gather를 안하는 등의 일이 일어날 수 있습니다.

**그럼 이거 왜 쓸까요? 결국 전체 파라미터세트가 allreduce해서 이동해야하는건 똑같지 않나요?**

앞서 말했듯 결국 비동기적으로 통신에 '적은'부담이 지속적으로 간다는 점에서 통신적으로 큰 이득이 있습니다.

그리고 무엇보다도 샤딩되었기 때문에 각 GPU에 올라가는 사이즈가 작습니다.

**단점?**

당연히 지속적으로 왔다갔다 해야하는 샤딩된 파라미터 때문에 느릴 수 있습니다. 다만 돌릴 수 없던 거대 모델을 쪼개서 돌릴 수 있다는 장점이 있겠죠?




-----

## FSDP2 사용 방법

### 모델 초기화

**서브모듈(submodules)에 `fully_shard` 적용하기:** DDP와 달리, FSDP2는 루트 모델뿐만 아니라 서브모듈에도 `fully_shard`를 적용해야 합니다. 정확히는 모델에다가 샤딩할 부분일 지정하고 DDP에 넣어야합니다. 

아래의 트랜스포머 예제에서는 각 레이어에 먼저 `fully_shard`를 적용한 다음, 루트 모델에 적용했습니다.

  * `layers[i]`의 forward 계산 중에는 나머지 레이어들이 샤딩되어 메모리 사용량을 줄입니다.
  * `fully_shard(model)` 내부에서 FSDP2는 `model.layers`의 파라미터를 제외하고, 성능 좋은 `all-gather` 및 `reduce-scatter`를 위해 나머지 파라미터를 파라미터 그룹으로 분류합니다.
  * `fully_shard`는 샤딩된 모델을 실제 학습 디바이스(예: `cuda`)로 이동시킵니다.

**명령어:** `torchrun --standalone -nnodes=1 --nproc_per_node 2 train.py`

```python
from torch.distributed.fsdp import fully_shard, FSDPModule

model = Transformer()
for layer in model.layers:
    fully_shard(layer)
fully_shard(model)

assert isinstance(model, Transformer)
assert isinstance(model, FSDPModule)

print(model)
# FSDPTransformer(
# (tok_embeddings): Embedding(...)
# ...
# (layers): 3 x FSDPTransformerBlock(...)
# (output): Linear(...)
# )

```

`print(model)`을 통해 래핑(wrapping)을 확인할 수 있습니다. `FSDPTransformer`는 `Transformer`와 `FSDPModule`의 공동 클래스(joint class)입니다. 단순하게 트랜스포머가 아니라 FSDP 화 되었음을 의미합니다.

`FSDPTransformerBlock`도 마찬가지입니다. 모든 FSDP2 공개 API는 `FSDPModule`을 이용하고 있습니다. 이 `FSDPModule` 의 메서드를 이용해 `model.unshard()`를 호출하여 `all-gather` 스케줄을 수동으로 제어할 수 있습니다. 자세한 내용은 아래의 "explicit prefetching"을 참조하세요.

**`model.parameters()`를 DTensor로 사용하기:** `fully_shard`는 랭크 간에 파라미터를 샤딩하고, `model.parameters()`를 일반 `torch.Tensor`에서 **DTensor**로 변환하여 샤딩된 파라미터를 나타냅니다. DTensor는 distributed tensor의 약자로 사실은 샤딩되어있어서 전체 set를 가지고 있지 않지만, 마치 sharding 되지 않은 full shape 의 텐서처럼 행동하는 파라미터를 위해 정의된 데이터클래스입니다. 실제로 저장되는 값은 없지만 공갈빵 맹키로 텅텅빈 형태로 대충 작동한다고 생각하시면 됩니다.

FSDP2는 기본적으로 0번째 차원(dim-0)에서 샤딩하므로 DTensor 배치는 `Shard(0)`이 됩니다. N개의 랭크가 있고 샤딩 전에 N개의 행을 가진 파라미터가 있다고 가정하면, 샤딩 후 각 랭크는 파라미터의 1개 행을 갖게 됩니다. `param.to_local()`을 사용해 샤딩된 파라미터를 검사할 수 있습니다.

```python
from torch.distributed.tensor import DTensor

for param in model.parameters():
    assert isinstance(param, DTensor)
    assert param.placements == (Shard(0),)
    # param.to_local()로 샤딩된 파라미터 검사

optim = torch.optim.Adam(model.parameters(), lr=1e-2)
```

옵티마이저는 `fully_shard`를 적용한 *후에* 생성됩니다. 모델과 옵티마이저의 `state dict`는 모두 DTensor로 표현됩니다.

**DTensor는 옵티마이저, 그래디언트 클리핑, 체크포인팅을 용이하게 합니다.**

  * `torch.optim.Adam`과 `torch.nn.utils.clip_grad_norm_`은 DTensor 파라미터에 대해 즉시(out of the box) 작동합니다. 이는 단일 디바이스 학습과 분산 학습 간의 코드를 일관되게 만듭니다.
  * DTensor 및 DCP API를 사용해 파라미터를 조작하여 전체 `state dict`를 얻을 수 있습니다. 자세한 내용은 아래 "State Dict" 섹션을 참조하세요. 분산 `state dict`의 경우 추가 통신 없이 체크포인트를 저장/로드할 수 있습니다. ([doc](https://pytorch.org/docs/stable/distributed.checkpoint.html))

-----

### 프리페칭(Prefetching)을 사용한 Forward/Backward

**명령어:** `torchrun --standalone -nnodes=1 --nproc_per_node 2 train.py`

```python
for _ in range(epochs):
    x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    loss = model(x).sum()
    loss.backward()
    optim.step()
    optim.zero_grad()
```

`fully_shard`는 forward/backward 훅(hook)을 등록하여 계산 전에 파라미터를 `all-gather`하고 계산 후에 파라미터를 `reshard`합니다. `all-gather`와 계산을 오버랩(overlap)하기 위해, FSDP2는 위 학습 루프에서 즉시 작동하는 **implicit prefetching**과, 고급 사용자가 `all-gather` 스케줄을 수동으로 제어할 수 있는 **explicit prefetching**을 제공합니다.

**implicit prefetching:** CPU 스레드는 레이어 `i` 이전에 `all-gather i`를 발행합니다. 레이어 `i` 계산이 기본 스트림(default stream)에서 일어나는 동안, `all-gather`는 자체 CUDA 스트림으로 큐에 들어갑니다. CPU 바운드 작업이 아닌 경우(예: 큰 배치 크기의 트랜스포머), `all-gather i+1`은 레이어 `i`의 계산과 오버랩될 수 있습니다. 암시적 프리페칭은 backward에서도 유사하게 작동하며, `all-gather`는 post-forward 순서의 역순으로 발행됩니다.

사용자는 먼저 implicit prefetching으로 시작하여 기본 성능을 이해하는 것을 권장합니다.

![prefetch](https://docs.pytorch.org/tutorials/_images/fsdp_implicit.png)



**explicit prefetching:** 사용자는 `set_modules_to_forward_prefetch`로 forward 순서를, `set_modules_to_backward_prefetch`로 backward 순서를 지정할 수 있습니다. 아래 코드에서 보듯이, CPU 스레드는 레이어 `i`에서 `all-gather i+1` 및 `i+2`를 발행합니다.

explicit prefetching은 다음 상황에서 잘 작동합니다.

  * **CPU 바운드 작업:** implicit 프리페칭을 사용하면, 레이어 `i`의 커널이 실행될 때 CPU 스레드가 너무 느려서 레이어 `i+1`의 `all-gather`를 발행하지 못할 수 있습니다. 레이어 `i`의 forward를 실행하기 전에 `all-gather i+1`을 명시적으로 발행해야 합니다.
  * **2개 이상의 레이어 프리페칭:** implicit 프리페칭은 메모리 사용량을 최소로 유지하기 위해 **한 번에 다음 한 레이어만** `all-gather`합니다. explicit 프리페칭을 사용하면 한 번에 여러 레이어를 `all-gather`하여 메모리 사용량이 증가하는 대신 더 나은 성능을 얻을 수 있습니다. (코드의 `layers_to_prefetch` 참조)
  * **첫 번째 `all-gather`를 더 일찍 발행하기:** implicit 프리페칭은 `model(x)`를 호출하는 시점에 발생합니다. 첫 번째 `all-gather`가 노출됩니다(오버랩되지 않음). `model.unshard()`를 명시적으로 더 일찍 호출하여 첫 번째 `all-gather`를 더 일찍 발행할 수 있습니다.

**명령어:** `torchrun --nproc_per_node 2 train.py --explicit-prefetching`

```python
num_to_forward_prefetch = 2
for i, layer in enumerate(model.layers):
    if i >= len(model.layers) - num_to_forward_prefetch:
        break
    layers_to_prefetch = [
        model.layers[i + j] for j in range(1, num_to_forward_prefetch + 1)
    ]
    layer.set_modules_to_forward_prefetch(layers_to_prefetch)

num_to_backward_prefetch = 2
for i, layer in enumerate(model.layers):
    if i < num_to_backward_prefetch:
        continue
    layers_to_prefetch = [
        model.layers[i - j] for j in range(1, num_to_backward_prefetch + 1)
    ]
    layer.set_modules_to_backward_prefetch(layers_to_prefetch)

for _ in range(epochs):
    # 첫 번째 all-gather를 더 일찍 트리거
    # 이는 model(x) 전의 모든 계산과 all-gather를 오버랩시킴
    model.unshard()
    
    x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    loss = model(x).sum()
    loss.backward()
    optim.step()
    optim.zero_grad()
```



-----

### 혼합 정밀도(Mixed Precision) 활성화 (optional)

FSDP2는 학습 속도를 높이기 위해 유연한 혼합 정밀도 정책(`MixedPrecisionPolicy`)을 제공합니다. 일반적인 사용 사례는 다음과 같습니다.

  * Forward/backward 계산을 위해 `float32` 파라미터를 `bfloat16`으로 캐스팅 (`param_dtype=torch.bfloat16` 참조)
  * 정확도를 보존하기 위해 `reduce-scatter`를 위해 그래디언트를 `float32`로 업캐스팅 (`reduce_dtype=torch.float32` 참조)

`torch.amp`와 비교할 때, FSDP2 혼합 정밀도는 다음과 같은 이점이 있습니다.

  * **성능 좋고 유연한 파라미터 캐스팅:** `FSDPModule` 내의 모든 파라미터는 모듈 경계(forward/backward 전후)에서 함께 캐스팅됩니다. 각 레이어마다 다른 혼합 정밀도 정책을 설정할 수 있습니다. (예: 처음 몇 개 레이어는 `float32`, 나머지 레이어는 `bfloat16`)
  * **`float32` 그래디언트 리덕션 (reduce-scatter):** 그래디언트는 랭크마다 크게 다를 수 있습니다. `float32`로 그래디언트를 리덕션하는 것은 수치 안정성(numerics)에 중요할 수 있습니다.

**명령어:** `torchrun --nproc_per_node 2 train.py --mixed-precision`

```python
from torch.distributed.fsdp.api import MixedPrecisionPolicy

model = Transformer(model_args)
fsdp_kwargs = {
    "mp_policy": MixedPrecisionPolicy(
        param_dtype=torch.bfloat16,
        reduce_dtype=torch.float32,
    )
}
for layer in model.layers:
    fully_shard(layer, **fsdp_kwargs)
fully_shard(model, **fsdp_kwargs)

# 샤딩된 파라미터는 float32
for param in model.parameters():
    assert param.dtype == torch.float32

# 샤딩되지 않은 파라미터는 bfloat16
model.unshard()
for param in model.parameters(recurse=False):
    assert param.dtype == torch.bfloat16
model.reshard()

# 옵티마이저 상태는 float32
optim = torch.optim.Adam(model.parameters(), lr=1e-2)

# 학습 루프
# ...
```



-----

### DTensor를 사용한 그래디언트 클리핑 및 옵티마이저

**명령어:** `torchrun --nproc_per_node 2 train.py`

```python
# 옵티마이저는 DTensor 모델 파라미터를 기반으로 생성됨
optim = torch.optim.Adam(model.parameters(), lr=1e-2)

for _ in range(epochs):
    x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    loss = model(x).sum()
    loss.backward()
    
    # 그래디언트 클리핑
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
    
    optim.step()
    optim.zero_grad()
```

옵티마이저는 모델에 `fully_shard`를 적용한 후에 초기화되며, DTensor인 `model.parameters()`에 대한 참조를 유지합니다. 그래디언트 클리핑의 경우, `torch.nn.utils.clip_grad_norm_`이 DTensor 파라미터에 대해 작동합니다. 텐서 연산은 DTensor 내부에서 올바르게 디스패치되어 랭크 간에 부분 텐서(partial tensor)를 통신하여 단일 디바이스 시맨틱(semantic)을 보존합니다.

-----

### DTensor API를 사용한 State Dicts

여기서는 전체 `state dict`를 로드하기 위해 DTensor `state dict`로 변환하는 방법과, 저장을 위해 다시 전체 `state dict`로 변환하는 방법을 보여줍니다.

**명령어:** `torchrun --nproc_per_node 2 train.py`

  * 처음 실행 시: 모델과 옵티마이저를 위한 체크포인트를 생성합니다.
  * 두 번째 실행 시: 이전 체크포인트에서 로드하여 학습을 재개합니다.

**State Dict 로드하기:** `meta` 디바이스에서 모델을 초기화하고 `fully_shard`를 호출하여 `model.parameters()`를 `torch.Tensor`에서 `DTensor`로 변환합니다. `torch.load`로 전체 `state dict`를 읽은 후, `distribute_tensor`를 호출하여 `model.state_dict()`의 동일한 배치(placements)와 디바이스 메시(device mesh)를 사용해 `torch.Tensor`를 `DTensor`로 변환할 수 있습니다. 마지막으로 `model.load_state_dict`를 호출하여 DTensor `state dict`를 모델에 로드합니다.

```python
from torch.distributed.tensor import distribute_tensor

# mmap=True는 CPU 메모리 사용량을 줄임
full_sd = torch.load(
    "checkpoints/model_state_dict.pt",
    mmap=True,
    weights_only=True,
    map_location='cpu',
)
meta_sharded_sd = model.state_dict()
sharded_sd = {}
for param_name, full_tensor in full_sd.items():
    sharded_meta_param = meta_sharded_sd.get(param_name)
    sharded_tensor = distribute_tensor(
        full_tensor,
        sharded_meta_param.device_mesh,
        sharded_meta_param.placements,
    )
    sharded_sd[param_name] = nn.Parameter(sharded_tensor)

# `meta` 텐서에는 `copy_`를 호출할 수 없으므로 `assign=True` 사용
model.load_state_dict(sharded_sd, assign=True)
```

**State Dict 저장하기:** `model.state_dict()`는 DTensor `state dict`를 반환합니다. `full_tensor()`를 호출하여 DTensor를 `torch.Tensor`로 변환할 수 있습니다. 내부적으로 이는 랭크 간 `all-gather`를 발행하여 샤딩되지 않은 파라미터를 `torch.Tensor`로 가져옵니다. 랭크 0의 경우, `full_param.cpu()`는 샤딩되지 않은 파라미터로 인한 GPU 메모리 피크를 피하기 위해 텐서를 하나씩 CPU로 오프로드합니다.

```python
sharded_sd = model.state_dict()
cpu_state_dict = {}
for param_name, sharded_param in sharded_sd.items():
    full_param = sharded_param.full_tensor()
    if torch.distributed.get_rank() == 0:
        cpu_state_dict[param_name] = full_param.cpu()
    else:
        del full_param
torch.save(cpu_state_dict, "checkpoints/model_state_dict.pt")
```

옵티마이저 `state dict`도 비슷하게 작동합니다 ([코드](https://www.google.com/search?q=https://github.com/pytorch/examples/blob/main/distributed/FSDP/FSDP_example.py%23L425)). 사용자는 위 DTensor 스크립트를 커스터마이징하여 서드파티(3rd party) 체크포인트와 연동할 수 있습니다.

커스터마이징이 필요 없다면, DCP API를 직접 사용하여 단일 노드 및 다중 노드 학습을 모두 지원할 수 있습니다.

-----

### DCP API를 사용한 State Dict

**명령어:** `torchrun --nproc_per_node 2 train.py --dcp-api`

  * 처음 실행 시: 모델과 옵티마이저를 위한 체크포인트를 생성합니다.
  * 두 번째 실행 시: 이전 체크포인트에서 로드하여 학습을 재개합니다.

**State Dict 로드하기:** `set_model_state_dict`를 사용해 FSDP2 모델에 전체 `state dict`를 로드할 수 있습니다. `broadcast_from_rank0=True`로 설정하면, 랭크 0에서만 전체 `state dict`를 로드하여 CPU 메모리 피크를 피할 수 있습니다. DCP가 텐서를 샤딩하고 다른 랭크로 브로드캐스트합니다.

```python
from torch.distributed.checkpoint.state_dict import (
    set_model_state_dict,
    StateDictOptions
)

set_model_state_dict(
    model=model,
    model_state_dict=full_sd,
    options=StateDictOptions(
        full_state_dict=True,
        broadcast_from_rank0=True,
    ),
)
```

**State Dict 저장하기:** `get_model_state_dict`를 `full_state_dict=True` 및 `cpu_offload=True`와 함께 사용하면 텐서를 `all-gather`하고 CPU로 오프로드합니다. 이는 DTensor API와 유사하게 작동합니다.

```python
from torch.distributed.checkpoint.state_dict import (
    get_model_state_dict,
    StateDictOptions
)

model_state_dict = get_model_state_dict(
    model=model,
    options=StateDictOptions(
        full_state_dict=True,
        cpu_offload=True,
    )
)
torch.save(model_state_dict, "model_state_dict.pt")
```

옵티마이저 `state dict` 로드 및 저장은 [pytorch/examples](https://www.google.com/search?q=https://github.com/pytorch/examples/blob/main/distributed/FSDP/FSDP_example.py)의 `set_optimizer_state_dict`와 `get_optimizer_state_dict`를 참조하세요.

